In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, log_loss, matthews_corrcoef, accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

# Load dataset
file_path = "C://Users//Admin//Downloads//DDNL//Big-5.csv"
dataset = pd.read_csv(file_path, delimiter=",", low_memory=False)

# Sample data setup (replace with real dataset columns)
X = np.random.rand(1000, 10)
y = np.random.randint(0, 3, 1000)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# One-hot encode labels
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)
num_classes = y_train_cat.shape[1]

# Data augmentation (simulated by noise addition for tabular data)
def augment_data(X, y, factor=2):
    X_aug, y_aug = [X], [y]
    for _ in range(factor - 1):
        noise = np.random.normal(0, 0.01, X.shape)
        X_aug.append(X + noise)
        y_aug.append(y)
    return np.vstack(X_aug), np.vstack(y_aug)

X_train_aug, y_train_cat_aug = augment_data(X_train, y_train_cat)

# Regularization configurations
results = []
regularizations = {
    "L1": l1(0.01),
    "L2": l2(0.01),
    "Dropout": None,
    "L1_L2": l1_l2(l1=0.005, l2=0.005)
}

for name, reg in regularizations.items():
    print(f"Training with {name} regularization...")
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=reg))
    if name == "Dropout":
        model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    model.fit(X_train_aug, y_train_cat_aug, epochs=30, batch_size=32, validation_split=0.2, callbacks=[early_stop], verbose=0)

    y_pred_probs = model.predict(X_test)
    y_pred = np.argmax(y_pred_probs, axis=1)

    auc_roc = roc_auc_score(y_test_cat, y_pred_probs, multi_class='ovr')
    logloss = log_loss(y_test_cat, y_pred_probs)
    mcc = matthews_corrcoef(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    results.append([name, auc_roc, logloss, mcc, accuracy, precision, recall, f1])

# Save results
results_df = pd.DataFrame(results, columns=["Regularization", "AUC-ROC", "Log Loss", "MCC", "Accuracy", "Precision", "Recall", "F1-Score"])
results_df.to_csv("C://Users//Admin//Downloads//DDNL//model1_performance.csv", index=False)
print("Results saved to C://Users//Admin//Downloads//DDNL//model1_performance.csv")


Training with L1 regularization...


C:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Training with L2 regularization...


C:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Training with Dropout regularization...


C:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Training with L1_L2 regularization...


C:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Results saved to C://Users//Admin//Downloads//DDNL//model1_performance.csv


In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss, matthews_corrcoef
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# Load dataset
file_path = "C://Users//Admin//Downloads//DDNL//Big-5.csv"
dataset = pd.read_csv(file_path)

# Sample data setup (replace with real dataset columns)
X = np.random.rand(1000, 10)  # 1000 samples, 10 features
y = np.random.randint(0, 3, 1000)  # 3 classes

# Preprocessing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape input for RNN [samples, timesteps, features]
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# One-hot encode targets
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)
num_classes = y_train_cat.shape[1]

# Define RNN model
model = Sequential()
model.add(SimpleRNN(64, activation='tanh', input_shape=(1, X_train.shape[2])))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(X_train, y_train_cat, epochs=30, batch_size=32, validation_split=0.2, callbacks=[early_stop], verbose=1)

# Evaluation
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

auc_roc = roc_auc_score(y_test_cat, y_pred_probs, multi_class='ovr')
logloss = log_loss(y_test_cat, y_pred_probs)
mcc = matthews_corrcoef(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Results
results = pd.DataFrame([['RNN', auc_roc, logloss, mcc, accuracy, precision, recall, f1]],
                       columns=["Model", "AUC-ROC", "Log Loss", "MCC", "Accuracy", "Precision", "Recall", "F1-Score"])
results.to_csv("C://Users//Admin//Downloads//DDNL//rnn_model_performance.csv", index=False)
print("RNN model performance saved to C://Users//Admin//Downloads//DDNL//rnn_model_performance.csv")


C:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.3216 - loss: 1.2053 - val_accuracy: 0.3812 - val_loss: 1.0969
Epoch 2/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3275 - loss: 1.1153 - val_accuracy: 0.3812 - val_loss: 1.0895
Epoch 3/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3892 - loss: 1.0913 - val_accuracy: 0.3562 - val_loss: 1.0940
Epoch 4/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4185 - loss: 1.0783 - val_accuracy: 0.3125 - val_loss: 1.1015
Epoch 5/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4184 - loss: 1.0714 - val_accuracy: 0.2937 - val_loss: 1.1032
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
RNN model performance saved to C://Users//Admin//Downloads//DDNL//rnn_model_performance.csv


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss, matthews_corrcoef
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Bidirectional
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# Load dataset
file_path = "C://Users//Admin//Downloads//DDNL//Big-5.csv"
dataset = pd.read_csv(file_path)

# Sample data setup (replace with real dataset columns)
X = np.random.rand(1000, 10)  # 1000 samples, 10 features
y = np.random.randint(0, 3, 1000)  # 3 classes

# Preprocessing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape input for RNN [samples, timesteps, features]
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# One-hot encode targets
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)
num_classes = y_train_cat.shape[1]

# Define Bidirectional RNN model
model = Sequential()
model.add(Bidirectional(SimpleRNN(64, activation='tanh'), input_shape=(1, X_train.shape[2])))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(X_train, y_train_cat, epochs=30, batch_size=32, validation_split=0.2, callbacks=[early_stop], verbose=1)

# Evaluation
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

auc_roc = roc_auc_score(y_test_cat, y_pred_probs, multi_class='ovr')
logloss = log_loss(y_test_cat, y_pred_probs)
mcc = matthews_corrcoef(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Results
results = pd.DataFrame([['Bidirectional RNN', auc_roc, logloss, mcc, accuracy, precision, recall, f1]],
                       columns=["Model", "AUC-ROC", "Log Loss", "MCC", "Accuracy", "Precision", "Recall", "F1-Score"])
results.to_csv("C://Users//Admin//Downloads//DDNL//bidir_rnn_model_performance.csv", index=False)
print("Bidirectional RNN model performance saved to C://Users//Admin//Downloads//DDNL//bidir_rnn_model_performance.csv")

Epoch 1/30


C:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - accuracy: 0.3255 - loss: 1.1558 - val_accuracy: 0.3000 - val_loss: 1.1229
Epoch 2/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.3788 - loss: 1.1018 - val_accuracy: 0.3250 - val_loss: 1.1332
Epoch 3/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3981 - loss: 1.0834 - val_accuracy: 0.3250 - val_loss: 1.1361
Epoch 4/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3858 - loss: 1.0828 - val_accuracy: 0.2937 - val_loss: 1.1382
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step
Bidirectional RNN model performance saved to C://Users//Admin//Downloads//DDNL//bidir_rnn_model_performance.csv


In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss, matthews_corrcoef
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# Load dataset
file_path = "C://Users//Admin//Downloads//DDNL//Big-5.csv"
dataset = pd.read_csv(file_path)

# Sample data setup (replace with real dataset columns)
X = np.random.rand(1000, 10)  # 1000 samples, 10 features
y = np.random.randint(0, 3, 1000)  # 3 classes

# Preprocessing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape input for LSTM [samples, timesteps, features]
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# One-hot encode targets
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)
num_classes = y_train_cat.shape[1]

# Define LSTM model
model = Sequential()
model.add(LSTM(64, activation='tanh', input_shape=(1, X_train.shape[2])))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(X_train, y_train_cat, epochs=30, batch_size=32, validation_split=0.2, callbacks=[early_stop], verbose=1)

# Evaluation
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

auc_roc = roc_auc_score(y_test_cat, y_pred_probs, multi_class='ovr')
logloss = log_loss(y_test_cat, y_pred_probs)
mcc = matthews_corrcoef(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Results
results = pd.DataFrame([['LSTM', auc_roc, logloss, mcc, accuracy, precision, recall, f1]],
                       columns=["Model", "AUC-ROC", "Log Loss", "MCC", "Accuracy", "Precision", "Recall", "F1-Score"])
results.to_csv("C://Users//Admin//Downloads//DDNL//lstm_model_performance.csv", index=False)
print("LSTM model performance saved to C://Users//Admin//Downloads//DDNL//lstm_model_performance.csv")


Epoch 1/30


C:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.3388 - loss: 1.0986 - val_accuracy: 0.3750 - val_loss: 1.1024
Epoch 2/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3514 - loss: 1.0937 - val_accuracy: 0.3688 - val_loss: 1.1053
Epoch 3/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3756 - loss: 1.0911 - val_accuracy: 0.3438 - val_loss: 1.1075
Epoch 4/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3939 - loss: 1.0874 - val_accuracy: 0.3500 - val_loss: 1.1102
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
LSTM model performance saved to C://Users//Admin//Downloads//DDNL//lstm_model_performance.csv
